In [ ]:
import requests
import json
import pandas as pd

# Configurações da API e credenciais
base_url    = 'https://trf5.atlassian.net/rest/api/3/project'
issues_url  = 'https://trf5.atlassian.net/rest/api/3/search'
worklog_url = 'https://trf5.atlassian.net/rest/api/3/issue'
user = 'xxxxxxxxx'
pwd = '*******************'

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

In [4]:
# Faz a requisição para obter todos os projetos
response = requests.get(base_url, auth=(user, pwd), headers=headers)

print(f"response: {response}")

response: <Response [200]>


In [6]:
data = response.json()

# Cria uma lista para armazenar os dados do DataFrame
projects_list = []

# Itera sobre os projetos e coleta o ID e KEY
for project in data:
    project_id            = project.get('id', 'ID não encontrado')
    project_key           = project.get('key', 'Key não encontrada')
    project_name          = project.get('name', 'name não encontrada')
    project_category      = project.get('projectCategory')
    project_category_name = project_category['name'] if project_category else 'projectCategory não encontrado'
    project_type_key      = project.get('projectTypeKey', 'projectTypeKey não encontado')
    project_archived      = project.get('archived', 'False')
    project_archivedDate  = project.get('archivedDate', '')

    projects_list.append({
        'project_id':           project_id,
        'project_key':          project_key,
        'project_name':         project_name,
        'project_category_name': project_category_name,
        'project_type_key':     project_type_key,
        'project_archived':     project_archived,
        'project_archivedDate': project_archivedDate
        })

# Cria o DataFrame a partir da lista de projetos
df_projects = pd.DataFrame(projects_list)

# Imprimir o número de linhas do DataFrame de projetos
num_linhas_projetos = df_projects.shape[0]
print(f"O DataFrame de projetos tem {num_linhas_projetos} linhas.")

# Salva o DataFrame de projetos em um arquivo CSV
df_projects.to_csv('projects.csv', index=False)
print("DataFrame de projetos salvo em projects.csv")


O DataFrame de projetos tem 313 linhas.
DataFrame de projetos salvo em projects.csv


In [11]:
# Função para extrair o campo texto de uma estrutura de conteúdo
def extract_text_field(field):
    """Extract text from a content field."""
    try:
        content = field['content']
        if content:
            first_paragraph = content[0]
            paragraph_content = first_paragraph['content']
            if paragraph_content:
                first_text = paragraph_content[0]
                return first_text['text']
    except (KeyError, IndexError, TypeError):
        return 'Informação não encontrada'
    
# Função para listar as issues de um projeto dado seu Key e retornar um DataFrame
def listar_issues(project_key):
    api_url = f"{issues_url}?jql=project={project_key}"
    response = requests.get(api_url, auth=(user, pwd), headers=headers)

    if response.status_code == 200:
        data = response.json()
        total = data.get("total", 0)
        print(f"Total de issues para o projeto {project_key}: {total}")

        # Define o tamanho do lote para a paginação
        batch_size = 100

        all_issues = []

        # Pagina através dos resultados, pegando todas as issues
        for start_at in range(0, total, batch_size):
            query = {
                "jql": f"project={project_key}",
                "startAt": start_at,
                "maxResults": batch_size
            }
            response = requests.get(issues_url, headers=headers, params=query, auth=(user, pwd))
            if response.status_code == 200:
                data = response.json()
                issues = data.get('issues', [])

                all_issues.extend(issues)
                #print(f"Obtidas {len(issues)} issues, total acumulado: {len(all_issues)}")
            else:
                print(f"{response.status_code}: Erro na requisição de paginação. Projeto: {project_key}")
                #print(response.text)
                break

        #print(f"Total de issues obtidas para o projeto {project_key}: {len(all_issues)}")

        issues_list = []
        # Cria um DataFrame com as issues
        for issue in all_issues:
            fields          = issue.get('fields', {})
            
            # Extraindo a descrição
            description      = fields.get('description', {})
            description_text = extract_text_field(description)
            
            creator         = fields.get('creator')
            creatorName     = creator['displayName'] if creator else 'creator não encontrado'
            
            assignee         = fields.get('assignee')
            assigneeName     = assignee['displayName'] if assignee else 'assignee não encontrado'
            
            reporter         = fields.get('reporter')
            reporterName     = reporter['displayName'] if reporter else 'reporter não encontrado'
            
            issuetype        = fields.get('issuetype')
            issuetypeName    = issuetype['name'] if issuetype else 'issuetype não encontrado'
            
            priority         = fields.get('priority')
            priorityName     = priority['name'] if priority else 'priority não encontrado'
            
            status           = fields.get('status', {})
            statusName       = status['name'] if status else 'status não encontrado'
            status_category  = status.get('statusCategory', {})
            status_category_name = status_category.get('name', 'statusCategory name não encontrado')
                        
            resolution      = fields.get('resolution')
            resolution_name = resolution['name'] if resolution else 'resolution não encontrado'
            
            parent          = fields.get('parent')
            parentid        = parent['id'] if parent else 'parent id não encontrado'
            parentKey       = parent['key'] if parent else 'parent id não encontrado'
            
            subtasks        = fields.get('subtasks')
            subtask_ids     = [subtask['id'] for subtask in subtasks] if subtasks else ['subTask id não encontrado']
            subtasks_keys   = [subtask['key'] for subtask in subtasks] if subtasks else ['subTask keys não encontrado']          
            
            # Verificação de uma segunda ierarquia
            inward_issue_ids = []
            inward_issue_keys = []
            issuelinks      = fields.get('issuelinks',[])
            for link in issuelinks:
                inward_issue = link.get('inwardIssue')
                if inward_issue:
                    inward_issue_ids.append(inward_issue.get('id', 'id não encontrado'))
                    inward_issue_keys.append(inward_issue.get('key', 'key não encontrado'))

            #Campos customizados customfield[0]
            customfield_10020                 = fields.get('customfield_10020', [])
            if customfield_10020:
                cf_info                       = customfield_10020[0]
                customfield_10020id           = cf_info.get('id', 'sprint ID não encontrado')
                customfield_10020name         = cf_info.get('name', 'sprint name não encontrado')
                customfield_10020state        = cf_info.get('state', 'sprint state id não encontrado')
                customfield_10020startDate    = cf_info.get('startDate', 'sprint startDate id não encontrado')
                customfield_10020endDate      = cf_info.get('endDate', 'sprint endDate id não encontrado')
                customfield_10020completeDate = cf_info.get('completeDate', 'sprint completeDate id não encontrado')
            else:
                customfield_10020id           = 'sprint ID não encontrado'
                customfield_10020name         = 'sprint name não encontrado'
                customfield_10020state        = 'sprint state id não encontrado'
                customfield_10020startDate    = 'sprint startDate id não encontrado'
                customfield_10020endDate      = 'sprint endDate id não encontrado'
                customfield_10020completeDate = 'sprint completeDate id não encontrado'
            
            # Extraindo o Último Comentário
            customfield_10126                 = fields.get('customfield_10126', {})
            customfield_10126_desc_text       = extract_text_field(customfield_10126)
                
            customfield_10095                 = fields.get('customfield_10095',[])
            if customfield_10095:
                customfield_10095value        = customfield_10095['value']
            else:
                customfield_10095value        = 'departamento não encontrado'
            
            customfield_10074                 = fields.get('customfield_10074',[])
            if customfield_10074:
                customfield_10074value        = customfield_10074['value']
            else:
                customfield_10074value        = 'produto não encontrado'
                
            customfield_10100                 = fields.get('customfield_10100',[])
            if customfield_10100:
                customfield_10100value        = customfield_10100['value']
            else:
                customfield_10100value        = 'departamento não encontrado'
                
            customfield_10052                 = fields.get('customfield_10052',{})
            if customfield_10052:
                customfield_10052value        = customfield_10052['value']
            else:
                customfield_10052value        = 'grau não encontrado'
            
            # Extraindo o motivo
            customfield_10096                 = fields.get('customfield_10096', {})
            customfield_10096_desc_text       = extract_text_field(customfield_10096)
            
            # Extraindo o motivo
            customfield_10108                 = fields.get('customfield_10108', {})
            customfield_10108_desc_text       = extract_text_field(customfield_10108)
            
            customfield_10086                 = fields.get('customfield_10086',[])
            if customfield_10086:
                customfield_10086value        = customfield_10086['value']
            else:
                customfield_10086value        = 'Órgão não encontrado'
            
            customfield_10141                 = fields.get('customfield_10141',[])
            if customfield_10141:
                customfield_10141value        = customfield_10141['value']
            else:
                customfield_10141value        = 'Centro de Custo não encontrado'
                
            customfield_10142                 = fields.get('customfield_10142',[])
            if customfield_10142:
                customfield_10142value        = customfield_10142['value']
            else:
                customfield_10142value        = 'PAC não encontrado'
            
            customfield_10180                 = fields.get('customfield_10180',[])
            if customfield_10180:
                customfield_10180value        = customfield_10180['value']
            else:
                customfield_10180value        = 'PSL não encontrado'
            
            customfield_10181                 = fields.get('customfield_10181',[])
            if customfield_10181:
                customfield_10181value        = customfield_10181[0]['value']
            else:
                customfield_10181value        = 'PSL não encontrado'
            
            # Extraindo o motivo
            customfield_10170                 = fields.get('customfield_10170', {})
            customfield_10170_desc_text       = extract_text_field(customfield_10170)
            
            # Extraindo o motivo
            customfield_10171                 = fields.get('customfield_10171', {})
            customfield_10171_desc_text       = extract_text_field(customfield_10171)
            
            # Extraindo o motivo
            customfield_10172                 = fields.get('customfield_10172', {})
            customfield_10172_desc_text       = extract_text_field(customfield_10172)
            
            # Extraindo o motivo
            customfield_10173                 = fields.get('customfield_10173', {})
            customfield_10173_desc_text       = extract_text_field(customfield_10173)
            
            customfield_10174                 = fields.get('customfield_10174',[])
            if customfield_10174:
                customfield_10174value        = customfield_10174['value']
            else:
                customfield_10174value        = 'Plano Estratégico não encontrado'
            
            customfield_10175                 = fields.get('customfield_10175',[])
            if customfield_10175:
                customfield_10175value        = customfield_10175[0]['value']
            else:
                customfield_10175value        = 'Obj Estratégico não encontrado'
            
            customfield_10176                 = fields.get('customfield_10176',[])
            if customfield_10176:
                customfield_10176value        = customfield_10176[0]['value']
            else:
                customfield_10176value        = 'Metas Estratégico não encontrado'
            
            customfield_10177                 = fields.get('customfield_10177',[])
            if customfield_10177:
                customfield_10177value        = customfield_10177['value']
            else:
                customfield_10177value        = 'PDIT não encontrado'
            
            customfield_10178                 = fields.get('customfield_10178',[])
            if customfield_10178:
                customfield_10178value        = customfield_10178[0]['value']
            else:
                customfield_10178value        = 'Inic. PDTI não encontrado'
            
            customfield_10165                 = fields.get('customfield_10165',[])
            if customfield_10165:
                customfield_10165value        = customfield_10165['value']
            else:
                customfield_10165value        = 'Não'
            
            customfield_10166                 = fields.get('customfield_10166',[])
            if customfield_10166:
                customfield_10166value        = customfield_10166['value']
            else:
                customfield_10166value        = 'Não'
            
            
            # Extraindo o escopo
            customfield_10168                 = fields.get('customfield_10168', {})
            customfield_10168_desc_text       = extract_text_field(customfield_10168)
            
            # Extraindo o objetivo
            customfield_10169                 = fields.get('customfield_10169', {})
            customfield_10169_desc_text       = extract_text_field(customfield_10169)
                       
            customfield_10193                 = fields.get('customfield_10193',[])
            if customfield_10193:
                customfield_10193value        = customfield_10193['value']
            else:
                customfield_10193value        = 'Não associado'
            
            customfield_10194                 = fields.get('customfield_10194',[])
            if customfield_10194:
                customfield_10194value        = customfield_10194['value']
            else:
                customfield_10194value        = 'Área Reasponsável não informada'
            
            customfield_10197                 = fields.get('customfield_10197',[])
            if customfield_10197:
                customfield_10197value        = customfield_10197[0]['value']
            else:
                customfield_10197value        = 'Obj PEGP não informada'
            
            customfield_10198                 = fields.get('customfield_10198',[])
            if customfield_10198:
                customfield_10198value        = customfield_10198['value']
            else:
                customfield_10198value        = 'PEGP não informada'
            
            customfield_10199                 = fields.get('customfield_10199',[])
            if customfield_10199:
                customfield_10199value        = customfield_10199['value']
            else:
                customfield_10199value        = 'Abrangência não informada'
            
            # Extraindo o escopo negativo
            customfield_10200                 = fields.get('customfield_10200', {})
            customfield_10200_desc_text       = extract_text_field(customfield_10200)
             
            customfield_10234                 = fields.get('customfield_10234',[])
            if customfield_10234:
                customfield_10234value        = customfield_10234['value']
            else:
                customfield_10234value        = 'Não'
            
            customfield_10263                 = fields.get('customfield_10263',[])
            if customfield_10263:
                customfield_10263value        = customfield_10263['value']
            else:
                customfield_10263value        = 'Não'
            
            customfield_10264                 = fields.get('customfield_10264',[])
            if customfield_10264:
                customfield_10264value        = customfield_10264['value']
            else:
                customfield_10264value        = 'Não'
            
            issues_list.append({
                'Project ID':                 project_key,
                'Issue ID':                   issue.get('id', 'ID não encontrado'),
                'Issue Key':                  issue.get('key', 'Key não encontrada'),
                'Issue Summary':              fields.get('summary', 'Summary não encontrado'),
                'Issue description':          description_text,
                'Issue environment':          fields.get('environment', 'environment não encontrado'),
                'Issue created':              fields.get('created'),
                'Issue lastViewed':           fields.get('lastViewed'),
                'Issue updated':              fields.get('updated'),
                'Issue resolutiondate':       fields.get('resolutiondate'),
                'Issue duedate':              fields.get('duedate'),
                'Issue labels':               fields.get('labels'),
                'Issue components_ids':       fields.get('components_ids'),
                'Issue components_names':     fields.get('components_names'),
                'Issue fixVersions_ids':      fields.get('fixVersions_ids'),
                'Issue fixVersions_names':    fields.get('fixVersions_names'),
                'Issue versions_ids':         fields.get('versions_ids'),
                'Issue versions_names':       fields.get('versions_names'),
                'Issue creator_displayName':  creatorName,
                'Issue assignee_displayName': assigneeName,
                'Issue reporter_displayName': reporterName,
                'Issue issuetype_name':       issuetypeName,
                'Issue priority_name':        priorityName,
                'Issue status_name':          statusName,
                'Issue status_Categoria':     status_category_name,
                'Issue resolution_name':      resolution_name,
                'Issue parent_key':           parentKey,
                'Issue parent_id':            parentid,
                'Issue subtasks_ids':         subtask_ids,
                'Issue subtasks_keys':        subtasks_keys,
                'Issue issuelinks_ids':       ', '.join(inward_issue_ids) if inward_issue_ids else 'inwardIssue id não encontrado',
                'Issue issuelinks_keys':      ', '.join(inward_issue_keys) if inward_issue_keys else 'inwardIssue key não encontrado',
                
                'Issue aggregatetimespent':   fields.get('aggregatetimespent',''),
                'Issue aggregateprogress':    fields.get('aggregateprogress',''),
                'Issue aggregatetimeoriginalestimate': fields.get('aggregatetimeoriginalestimate',''),
                'Issue timeoriginalestimate': fields.get('timeoriginalestimate',''),
                'Issue timeestimate':         fields.get('timeestimate',''),
                
                #Campos Customizados
                'customfields_rank':                fields.get('customfield_10019',''),
                'CustomFields_startDate':           fields.get('customfield_10015',''),
                'CustomFields_endDate ':            fields.get('customfield_10035',''),
                'CustomFields_sprint_id ':          customfield_10020id,
                'CustomFields_sprint_name':         customfield_10020name,
                'CustomFields_sprint_state':        customfield_10020state,
                'CustomFields_sprint_startDate':    customfield_10020startDate,
                'CustomFields_sprint_endDate':      customfield_10020endDate,
                'CustomFields_sprint_completeDate': customfield_10020completeDate,
                'CustomFields_epic_pai ':           fields.get('customfield_10014',''),
                'CustomFields_epicLink':            fields.get('customfield_10011',''),
                'CustomFields_ultimo_comentario':   customfield_10126_desc_text,
                'CustomFields_StoryPoints':         fields.get('customfield_10041',''),
                'CustomFields_usuarios_apoiadores': fields.get('customfield_10128_displayName',''),
                'CustomFields_baseline_startdate':  fields.get('customfield_10036',''),
                'CustomFields_baseline_enddate':    fields.get('customfield_10037',''),
                'CustomFields_categoria':           customfield_10095value,
                'CustomFields_storyPoints_estimate': fields.get('customfield_10016',''),
                'CustomFields_produto':             customfield_10074value,
                'CustomFields_motivo':              customfield_10108_desc_text,
                'CustomFields_departamento':        customfield_10100value,
                'CustomFields_dataFaturamento':     fields.get('customfield_10101',''),
                'CustomFields_grauPrioridade':      customfield_10052value,
                'CustomFields_numeroExodus':        fields.get('customfield_10190',''),
                'CustomFields_PF_estimado':         fields.get('customfield_10091',''),
                'CustomFields_PF_realizado':        fields.get('customfield_10092',''),
                'CustomFields_justificativa':       customfield_10096_desc_text,
                'CustomFields_codigo_Externo':      fields.get('customfield_10097',''),
                'CustomFields_orgao':               customfield_10086value,
                'CustomFields_solicitante':         fields.get('customfield_10087',''),
                'CustomFields_corresponsavel':      fields.get('customfield_10089',''),
                'CustomFields_licoes_Aprendidas':   fields.get('customfield_10116',''),
                'CustomFields_observacoes':         fields.get('customfield_10117',''),
                'CustomFields_centrodecusto':       customfield_10141value,
                'CustomFields_pac':                 customfield_10142value,
                'CustomFields_pls':                 customfield_10180value,
                'CustomFields_ObjSustentavel':      customfield_10181value,
                'CustomFields_JustificativaProjeto': customfield_10170_desc_text,
                'CustomFields_EntregaPrevista':     customfield_10171_desc_text,
                'CustomFields_ResultadoEsperado':   customfield_10172_desc_text,
                'CustomFields_AbrangenciaImpacto':  customfield_10173_desc_text,
                'CustomFields_PlanoEstrategico':    customfield_10174value,
                'CustomFields_ObjEstrategico':      customfield_10175value,
                'CustomFields_MetasEstrategicas':   customfield_10176value,
                'CustomFields_PDTI':                customfield_10177value,
                'CustomFields_IniciativaPDTI':      customfield_10178value,
                'CustomFields_Estrategico':         customfield_10165value,
                'CustomFields_RedeInovacao':        customfield_10166value,
                'CustomFields_IdItemPAC':           fields.get('customfield_10167',''),
                'CustomFields_EscopoProjeto':       customfield_10168_desc_text,
                'CustomFields_ObjetivoProjeto':     customfield_10169_desc_text,
                'Customfields_Portfolio':           customfield_10193value,
                'Customfields_AreaResp':            customfield_10194value,
                'Customfields_ObjetivosPEGP':       customfield_10197value,
                'Customfields_PEGP':                customfield_10198value,
                'Customfields_AbrangenciaProjeto':  customfield_10199value,
                'Customfields_EscopoNegativo':      customfield_10200_desc_text,
                'Customfields_ProjetoPublico':      customfield_10234value,
                'Customfields_NumeroSEI':           fields.get('customfield_10235',''),
                'Customfields_Arquivar':            customfield_10263value,
                'Customfields_Programa':            customfield_10264value

            })

        df_issues = pd.DataFrame(issues_list)
        return df_issues
    else:
        print(f"Erro na requisição das issues para o projeto {project_key}: {response.status_code}")
        #print(response.text)
        return pd.DataFrame()

In [ ]:
# Lista para armazenar todos os DataFrames de issues
all_issues_dfs = []

# Iterar sobre cada projeto e listar as issues
df_issues = listar_issues("INAC")
if not df_issues.empty:
    all_issues_dfs.append(df_issues)
    
# Concatena todos os DataFrames de issues em um único DataFrame
if all_issues_dfs:
    df_all_issues = pd.concat(all_issues_dfs, ignore_index=True)

    # Imprimir o número de linhas do DataFrame de issues
    num_linhas_issues = df_all_issues.shape[0]
    print(f"O DataFrame de issues tem {num_linhas_issues} linhas.")

    # Salva o DataFrame de issues em um arquivo CSV
    df_all_issues.to_csv('issues.csv', index=False)
    print("DataFrame de issues salvo em issues.csv")
else:
    print("Nenhuma issue encontrada.")

Total de issues para o projeto INAC: 1945
O DataFrame de issues tem 1945 linhas.
DataFrame de issues salvo em issues.csv


In [17]:
# Função para listar os worklogs das issuese retornar um DataFrame
def listar_issues_worklog(issue_key):
    w_api_url = f"{worklog_url}/{issue_key}/worklog"
    response = requests.get(w_api_url, auth=(user, pwd), headers=headers)  
    
    if response.status_code == 200:
        data = response.json()   
    
        # Extraindo as informações
        time_spent_data = []

        for worklog in data['worklogs']:
           author          = worklog.get('author',[])
        
           if author:
               accountId   = author.get('accountId')
               displayName = author.get('displayName')
           else:
               accountId   = ''
               displayName = ''
               
           time_spent_data.append({    
               'work_accountId':   accountId,
               'work_displayName': displayName,
               'id_worklog':       worklog['id'],
               'started':          worklog['started'],
               'timeSpent':        worklog['timeSpent'],
               'timeSpentSeconds': worklog['timeSpentSeconds'],
               'issueId':          worklog['issueId']
            }) 
    
    df_issues_worklogs = pd.DataFrame(time_spent_data)
    # Imprimir o número de linhas do DataFrame 
    num_linhas_worklogs = df_issues_worklogs.shape[0]
    #print(f"O DataFrame de worklogs tem {num_linhas_worklogs} linhas.")
        
    return df_issues_worklogs

In [18]:
# Lista para armazenar todos os DataFrames de issues_worklog
all_issueswork_dfs = []

for issuekey in df_all_issues['Issue ID']:
    df_issues_worklog = listar_issues_worklog(issuekey)
    if not df_issues_worklog.empty:
        all_issueswork_dfs.append(df_issues_worklog)

if all_issueswork_dfs:
    df_all_issues_worklog = pd.concat(all_issueswork_dfs, ignore_index=True)

    # Imprimir o número de linhas do DataFrame de issues
    num_linhas_issues_worklog = df_all_issues_worklog.shape[0]
    print(f"O DataFrame de issues tem {num_linhas_issues_worklog} linhas.")

    # Salva o DataFrame de issues em um arquivo CSV
    df_all_issues_worklog.to_csv('issues_worklogs.csv', index=False)
    print("DataFrame de issues worklos salvo em issues_worklogs.csv")
else:
    print("Nenhuma issue worklog encontrada.")

O DataFrame de issues tem 6060 linhas.
DataFrame de issues worklos salvo em issues_worklogs.csv
